In [2]:
! pip3 install xgboost
! pip3 install category_encoders

     |████████████████████████████████| 157.5 MB 21 kB/s  eta 0:00:01     |██▊                             | 13.2 MB 12.0 MB/s eta 0:00:13     |██▉                             | 13.9 MB 12.0 MB/s eta 0:00:12     |█████▏                          | 25.3 MB 553 kB/s eta 0:03:59     |████████▏                       | 40.0 MB 1.7 MB/s eta 0:01:09     |███████████▍                    | 56.0 MB 1.5 MB/s eta 0:01:06     |████████████████                | 78.6 MB 1.5 MB/s eta 0:00:54     |██████████████████▉             | 92.9 MB 1.3 MB/s eta 0:00:49     |███████████████████████████████▉| 156.8 MB 2.0 MB/s eta 0:00:01


In [1]:
import numpy as np
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
import xgboost as xgb
import category_encoders
import pandas as pd
from sklearn.model_selection import GridSearchCV

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
df_train = pd.read_csv('data/train.csv')
df_train

In [ ]:
#Elimino columnas vacios o con datos que no sirven

train = df_train.drop(columns = ['Last_Activity','Actual_Delivery_Date','Price','Size','Product_Type','Brand'])
#Filtro solo los registros que esten en estado finalizado y los convierto en valores binarios
#Closed Won = 1 - Closed Lost = 0
train = train[(train['Stage'] == 'Closed Won') | (train['Stage'] == 'Closed Lost')]
train['Stage'] = train['Stage'].transform(func=lambda x : 1 if x=='Closed Won' else 0)
train
#ordeno por fecha de creacion de la oportunidad
train["Opportunity_Created_Date"] = pd.to_datetime(train["Opportunity_Created_Date"])
train.sort_values("Opportunity_Created_Date",inplace = True)


In [ ]:
target = train["Stage"].to_frame()

In [ ]:
features = train.loc[:,["Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved","Bureaucratic_Code_0_Approval",\
                       "Bureaucratic_Code_0_Approved","Submitted_for_Approval","ASP","ASP_(converted)","TRF",\
                        "Total_Amount","Total_Taxable_Amount","Stage"]].fillna(0)

features
#hay datos nan que hacemos??
#lo lleno de 0 momentaneamente

In [ ]:
Opp = train.loc[:,["Opportunity_ID","ASP","ASP_(converted)","TRF","Total_Amount","Total_Taxable_Amount"]].fillna(0).groupby("Opportunity_ID")
meanOpp = Opp.transform("mean").add_suffix("_Opp_Mean")
countOpp = Opp.transform("count").loc[:,["TRF"]].rename(columns={"TRF":"Opp_Item_Count"})
features = features.join(meanOpp)
features = features.join(countOpp)
features

## Mean encoding

#### Region

In [ ]:
tg = category_encoders.TargetEncoder(smoothing=True)
#Ajusto el codificador usando la variable categórica y el objetivo
tg.fit(train['Region'], train['Stage'])
#Aplico la codificacion a la columna Region
region_feature_encoded = tg.transform(train['Region'])
#Guardo la columna en el dataframe de entrenamiento
region = train.join(region_feature_encoded.add_suffix('_target_mean'))
region

#### Territory

In [ ]:
tg = category_encoders.TargetEncoder(smoothing=True)
#Ajusto el codificador usando la variable categórica y el objetivo
tg.fit(train['Territory'], train['Stage'])
#Aplico la codificacion a la columna Region
territory_feature_encoded = tg.transform(train['Territory'])
#Guardo la columna en el dataframe de entrenamiento
territory = train.join(territory_feature_encoded.add_suffix('_target_mean'))
territory.head()

#### Varios features con mean encoding

In [ ]:
columns_mean = ["Region","Territory","Bureaucratic_Code","Account_Type","ASP_Currency","Source ","Opportunity_Owner","Account_Owner"]

for column in columns_mean:
    tg_en = category_encoders.TargetEncoder(smoothing=True)
    #Ajusto el codificador usando la variable categórica y el objetivo
    tg_en.fit(train[column], train['Stage'])
    #Aplico la codificacion a la columna
    feature_encoded = tg_en.transform(train[column])
    #Guardo la columna en el dataframe de entrenamiento
    features = features.join(feature_encoded.add_suffix('_target_mean'))
    
features

## One Hot Encoding

In [ ]:
features["USD_ASP_Currency"] = train['ASP_Currency'].transform(func=lambda x : 1 if x=='USD' else 0)
features["EUR_ASP_Currency"] = train['ASP_Currency'].transform(func=lambda x : 1 if x=='EUR' else 0)
features["JPY_ASP_Currency"] = train['ASP_Currency'].transform(func=lambda x : 1 if x=='JPY' else 0)
features["AUD_ASP_Currency"] = train['ASP_Currency'].transform(func=lambda x : 1 if x=='AUD' else 0)
features["GBP_ASP_Currency"] = train['ASP_Currency'].transform(func=lambda x : 1 if x=='GBP' else 0)

features.head()

In [ ]:
features["Japan_Region"] = train['Region'].transform(func=lambda x : 1 if x=='Japan' else 0)
features["EMEA_Region"] = train['Region'].transform(func=lambda x : 1 if x=='EMEA' else 0)
features["Americas_Region"] = train['Region'].transform(func=lambda x : 1 if x=='Americas' else 0)
features["APAC_Region"] = train['Region'].transform(func=lambda x : 1 if x=='APAC' else 0)
features["Middle_East_Region"] = train['Region'].transform(func=lambda x : 1 if x=='Middle East' else 0)
features

## Binary Encoding

In [ ]:
columns = ["Region","Territory","Bureaucratic_Code","Account_Type","ASP_Currency","Source ","Opportunity_Owner","Account_Owner"]

for column in columns:
    bi_en = category_encoders.BinaryEncoder()
    #Ajusto el codificador usando la variable categórica y el objetivo
    bi_en.fit(train[column], train['Stage'])
    #Aplico la codificacion a la columna
    feature_encoded = bi_en.transform(train[column])
    #Guardo la columna en el dataframe de entrenamiento
    features = features.join(feature_encoded.add_suffix('_target_binary_enc'))
    
features

In [ ]:
#falta mas features basados en texto encodeados con tf-idf y otros
# fechas ?? esta ordenado por fecha creacion de oportunidad, hay que poner columna de mes año dia, etc
# analisis de los mejores features
#hace falta encodear todos??
#no sirven mucho los de one hot encoding
# falta la mezcla de precios montos total trf
# falta concatenar territorio y region y encodear
#falta concatenar moneda con otros
# vendedor y demas

In [ ]:
#features.to_csv('data/features.csv')
features.columns